In [9]:
from astropy.table import Table, join
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [25]:
# Import GZH catalog that Mel G. sent me
stripe82 = Table.read('gzh_sdss_intIDs.fits')
print len(stripe82)

# Import SDSS ancillary data from DR7 used with GZ2 galaxies
sdss = Table.read('../gzcodez/catalogs_GZ/zoo2AncillaryInfo.fits')

51861


### Match the GZH-stripe82 catalog with SDSS ancillary data

In [24]:
s82_sdss = join(stripe82, sdss, keys='OBJID')
print len(s82_sdss)

51861


### Select out low-z and low-smooth galaxies

In [54]:
non_nan = s82_sdss[~np.isnan(s82_sdss['REDSHIFT'])]
print len(stripe82) - len(non_nan), "subjects have no redshift in DR7"
print len(non_nan), "subjects have spectro-z"
print ""

sub_idx = np.where((non_nan['REDSHIFT']<=0.06) & 
                   (non_nan['t01_smooth_or_features_a01_smooth_weighted_fraction']<=0.7))
subsample = non_nan[sub_idx]
print len(subsample), "subjects in low-z & low-smooth sample"

14313 subjects have no redshift in DR7
37548 subjects have spectro-z

5543 subjects in low-z & low-smooth sample


### In the subsample, how many are clumpy?

In [39]:
clumpy_idx = np.where((subsample['t01_smooth_or_features_a02_features_or_disk_weighted_fraction'] >= 0.5 ) &
                      (subsample['t12_clumpy_a01_yes_weighted_fraction'] >= 0.5) &
                      (subsample['t12_clumpy_total_count'] >= 20) )
clumpy = subsample[clumpy_idx]
print len(clumpy), "subjects remaining"
print "Fraction of clumpy gals: ", float(len(clumpy))/float(len(subsample))

111 subjects remaining
Fraction of clumpy gals:  0.020025257081


### If we had redshifts for ALL galaxies, how many more clumpy? 

In [46]:
# no_z x subsample / total_with_z
addl_low_z_low_smooth = 14313*5543/37548

addl_clumpy = addl_low_z_low_smooth * 0.02
print addl_clumpy

42.24


### How many clumpy will we find in non-Stripe82 GZ2 galaxies?
### (that have spectroscopic z...)

In [66]:
# Read in GZ2 catalog 
# (already matched with other catalogs)
# Also, coadd-ed Stripe82 subjects have already been excluded
gz2 = Table.read('../GZExpress/SpaceWarps/analysis/GZ2ASSETS_NAIR_MORPH_MAIN_SDSS.fits')
print len(gz2)

295305


### Exclude Stripe82 subjects

In [68]:
gz2_non_stripe82 = gz2[gz2['stripe82']==0]
print len(gz2_non_stripe82)

273783


In [78]:
gz2_non_nan = gz2_non_stripe82[~np.isnan(gz2_non_stripe82['REDSHIFT'])]
print len(gz2_non_nan), "subjects in non-S82 with spec-z"

gz2_sub_idx = np.where((gz2_non_nan['REDSHIFT']<=0.06) &
                       (gz2_non_nan['t01_smooth_or_features_a01_smooth_weighted_fraction']<=0.7) )

gz2_subsample = gz2_non_stripe82[gz2_sub_idx]
print len(gz2_subsample), "subjects GZ2 low-z, low-smooth subsample"

234963 subjects in non-S82 with spec-z
43762 subjects GZ2 low-z, low-smooth subsample


In [79]:
GZ2_clumpy = 0.02 * len(gz2_subsample)

print GZ2_clumpy

875.24
